In [1]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [2]:
# #----测试-----
# print(critics['Lisa Rose'],'\n')

# for i in critics['Lisa Rose']:
#     print(i)
# print('\n')
# print(sum(critics['Lisa Rose'][i] for i in  critics['Lisa Rose']))
# print('\n')
# si = {}
# for i in critics['Lisa Rose']: 
#     if i in critics['Gene Seymour']: si[i]=1
# print(si,'\n')
# sum_of_squares=sum([pow(critics['Lisa Rose'][i]-critics['Gene Seymour'][i],2) 
#                     for i in critics['Lisa Rose'] if i in critics['Gene Seymour']])
# print(1/(1+sum_of_squares),'\n')


In [3]:
#----------------------------------------------------------------
#--------------user-based collaborative filtering----------------
#----------------------------------------------------------------
from math import sqrt

# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):   #使用欧几里得距离计算相似度
    # Get the list of shared_items
    si={}   #创建一个空字典用来存放共同特征
    for i in prefs[person1]: 
        if i in prefs[person2]: 
            si[i]=1   #共同拥有的特征（即共同看过并评分的电影）标记为1
            
    # if they have no ratings in common, return 0
    if len(si)==0: 
        return 0
    else:
        #核心代码
        sum_of_squares=sum([pow(prefs[person1][i]-prefs[person2][i],2) 
                      for i in prefs[person1] if i in prefs[person2]])#对于person1和person2共同拥有的特征i（即共同看过并评分
        #的电影），计算他们之间所有的（评分）差的平方和
    return 1/(1+sqrt(sum_of_squares))#加1避免出现0，取倒数使得值的范围在【0，1】之间

In [4]:
#-----测试------
sim_distance(critics,'Lisa Rose','Jack Matthews')

0.3405424265831667

In [8]:
def sim_pearson(prefs,p1,p2):  #使用皮尔逊相关系数计算相似度
    #得到双方都曾评价过的影片列表
    si={} 
    for item in prefs[p1]: 
        if item in prefs[p2]: si[item]=1
    n=len(si)
    # if they are no ratings in common, return 0
    if n==0: 
        return 0
    else:
        # 对所有偏好求和
        sum1=sum([prefs[p1][it] for it in si])
        sum2=sum([prefs[p2][it] for it in si])
        # 求平方和
        sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
        sum2Sq=sum([pow(prefs[p2][it],2) for it in si])	
        # 求乘积之和
        pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
        
        # 利用皮尔逊相关系数公式，求皮尔逊相关系数
        num=pSum-(sum1*sum2/n)
        den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
        if den==0: 
            return 0 #如果分母为零则返回0避免出错
        else:
            r=num/den
        return r

In [9]:
#-----测试-----
sim_pearson(critics,'Lisa Rose','Jack Matthews')

0.7470178808339965

In [10]:
def topMatches(prefs,person,n=5,similarity=sim_pearson): #注意到n和similarity赋了初值，意味着n和similarity在函数调用时可以缺省
    scores=[(similarity(prefs,person,otherPerson),otherPerson) 
            for otherPerson in prefs if otherPerson!=person] #将函数传入的这个人与字典prefs中的其他人分别比较（他们之间的相似度）
    scores.sort()#对列表socre进行排序（sort函数默认升序排列）
    scores.reverse()
    return scores[0:n]

In [11]:
#----测试----
topMatches(critics,'Lisa Rose')#可以缺省n和similarity，根据输出即可知道谁与Lisa Rose品味最相近

[(0.9912407071619299, 'Toby'),
 (0.7470178808339965, 'Jack Matthews'),
 (0.5940885257860044, 'Mick LaSalle'),
 (0.5669467095138396, 'Claudia Puig'),
 (0.40451991747794525, 'Michael Phillips')]

In [12]:
def getRecommendations(prefs,person,similarity=sim_pearson): #利用他人对影片评价的加权（权值依据前面计算的相似度赋值）平均得分来为某人提供推荐
    totals={}
    simSums={}
    for other in prefs:
        if other==person: 
            continue #自己不用与自己比较，忽略自己与自己的相似度计算
        else:
            sim=similarity(prefs,person,other)#计算自己与其他所有人的相似度
            
        if sim<=0:
            continue #相似度于等于零的忽略
        else:
            for item in prefs[other]:
                if item not in prefs[person] or prefs[person][item]==0:  #只对自己还未看过的影片进行评分推荐
                    # Similarity * Score
                    totals.setdefault(item,0)
                    totals[item]+=prefs[other][item]*sim
                    # Sum of similarities
                    simSums.setdefault(item,0)
                    simSums[item]+=sim
    #建立归一化列表
    rankings=[(total/simSums[item],item) for item,total in totals.items()]
    
    # Return the sorted list
    rankings.sort()
    rankings.reverse()
    return rankings

In [13]:
#----测试-----
getRecommendations(critics,'Toby')

[(3.3477895267131017, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]

In [23]:
#----------------------------------------------------------------
#--------------item-based collaborative filtering----------------
#----------------------------------------------------------------
def transformPrefs(prefs):#首先将用户和物品置换位置
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            # Flip item and person
            result[item][person]=prefs[person][item]
    return result
# #------测试-------
# transformPrefs(critics)

In [28]:
def calculateSimilarItems(prefs,n=10):#计算物品间的相似性
    result={}
    itemPrefs=transformPrefs(prefs)
    c=0
    for item in itemPrefs:
        #针对大数据集更新状态变量
        c+=1
        if c%100==0: print ("%d / %d")% (c,len(itemPrefs))
            
        #利用前面的函数计算两个「物品」之间的相似度
        scores=topMatches(itemPrefs,item,n=n,similarity=sim_distance)
        result[item]=scores
    return result
# #------测试-----------
# calculateSimilarItems(critics)

In [30]:
def getRecommendedItems(prefs,itemMatch,user):
    userRatings=prefs[user]
    scores={}
    totalSim={}
    #循环遍历当前用户评分的物品
    for (item,rating) in userRatings.items():
        # 循环遍历与当前物品相近的物品
        for (similarity,item2) in itemMatch[item]:
            # 用户已经对当前物品评价则忽略
            if item2 in userRatings: continue
            # 评价值与相似度加权和
            scores.setdefault(item2,0)
            scores[item2]+=similarity*rating
            # 全部相似度之和
            totalSim.setdefault(item2,0)
            totalSim[item2]+=similarity

    rankings=[(score/totalSim[item],item) for item,score in scores.items( )]
    rankings.sort()
    rankings.reverse()
    return rankings